In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load
import time

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [4]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer, InterpretableMultiHeadAttention
import libs.utils as utils

# LOAD DATA AND EXTRACT TESTSET

In [5]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_wind_mm_cop.csv'

In [6]:
raw_data: DataFrame = pd.read_csv(file_path)

In [7]:
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2016-12-31 23:00:00,225.000,-0.902831,4.980909,1028.935552,983.718765,0.000000e+00,1.381208,2.254668,...,0,UP_MPNTLCDMRN_1,23,31,5,12,UP_MPNTLCDMRN_1,0.0,5,23
1,UP_MPNTLCDMRN_1,2017-01-01 00:00:00,65.625,-0.783022,4.890773,1028.693079,983.524042,-1.734723e-15,1.278206,2.224647,...,0,UP_MPNTLCDMRN_1,0,1,6,1,UP_MPNTLCDMRN_1,1.0,6,0
2,UP_MPNTLCDMRN_1,2017-01-01 01:00:00,31.875,-0.565645,4.807044,1028.156679,983.034144,-1.734723e-15,1.211802,2.130597,...,0,UP_MPNTLCDMRN_1,1,1,6,1,UP_MPNTLCDMRN_1,2.0,6,1
3,UP_MPNTLCDMRN_1,2017-01-01 02:00:00,0.000,-0.331075,4.911959,1028.162779,983.062126,-1.734723e-15,1.017035,1.846413,...,0,UP_MPNTLCDMRN_1,2,1,6,1,UP_MPNTLCDMRN_1,3.0,6,2
4,UP_MPNTLCDMRN_1,2017-01-01 03:00:00,0.000,-0.668040,4.464566,1027.616920,982.551146,-1.734723e-15,0.715094,1.450024,...,0,UP_MPNTLCDMRN_1,3,1,6,1,UP_MPNTLCDMRN_1,4.0,6,3


In [8]:
raw_data['time'] = raw_data['time'].astype('datetime64[s]')

In [9]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [10]:
formatter: SorgeniaFormatter = config.make_data_formatter()

In [11]:
data_csv_path: str = config.data_csv_path

SPLIT DATA

In [12]:
train, valid, test = formatter.split_data(raw_data)

Setting scalers with training data...


In [13]:
test['time'].unique()[168]

numpy.datetime64('2020-08-13T23:00:00.000000000')

PICK OBSERVED DATA

In [14]:
forecast_time: str = '2020-08-13 22:00:00'

In [15]:
obs: DataFrame = test[test['time']<= forecast_time]

In [16]:
obs

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
31536,UP_MPNTLCDMRN_1,2020-08-06 23:00:00,-0.529162,0.866993,0.239003,-0.048342,-1.274784,-0.191301,-0.733072,-0.050166,...,1314,UP_MPNTLCDMRN_1,1.661345,6,0.000959,8,0,2.726227,3,23
31537,UP_MPNTLCDMRN_1,2020-08-07 00:00:00,-0.576865,0.866993,0.197319,-0.101425,-1.304034,-0.191301,-0.733072,-0.127810,...,1314,UP_MPNTLCDMRN_1,-1.661381,7,0.500952,8,0,2.726368,4,0
31538,UP_MPNTLCDMRN_1,2020-08-07 01:00:00,-0.642457,0.904884,0.127844,-0.119119,-1.304034,-0.191301,-0.792745,-0.321922,...,1314,UP_MPNTLCDMRN_1,-1.516914,7,0.500952,8,0,2.726510,4,1
31539,UP_MPNTLCDMRN_1,2020-08-07 02:00:00,-0.666309,0.866993,0.072265,-0.136814,-1.313784,-0.191301,-0.792745,-0.399567,...,1314,UP_MPNTLCDMRN_1,-1.372448,7,0.500952,8,0,2.726651,4,2
31540,UP_MPNTLCDMRN_1,2020-08-07 03:00:00,-0.691651,0.848048,0.044475,-0.136814,-1.323534,-0.191301,-0.792745,-0.360744,...,1314,UP_MPNTLCDMRN_1,-1.227982,7,0.500952,8,0,2.726793,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242083,UP_PRCLCMINEO_1,2020-08-13 18:00:00,-0.571595,1.140945,1.262832,0.113918,-0.178818,-0.159763,0.552657,0.240369,...,1320,UP_PRCLCMINEO_1,0.939014,13,0.000959,8,6,2.749270,3,18
242084,UP_PRCLCMINEO_1,2020-08-13 19:00:00,-0.663904,1.084093,1.075600,0.150009,-0.125213,-0.159763,0.297430,0.155619,...,1320,UP_PRCLCMINEO_1,1.083480,13,0.000959,8,6,2.749412,3,19
242085,UP_PRCLCMINEO_1,2020-08-13 20:00:00,-0.694163,1.008289,0.963260,0.222191,-0.071608,-0.159763,0.042203,-0.056255,...,1320,UP_PRCLCMINEO_1,1.227946,13,0.000959,8,6,2.749553,3,20
242086,UP_PRCLCMINEO_1,2020-08-13 21:00:00,-0.684299,1.027240,0.875885,0.294373,-0.018003,-0.159763,-0.021604,-0.056255,...,1320,UP_PRCLCMINEO_1,1.372413,13,0.000959,8,6,2.749694,3,21


In [17]:
assert obs.shape[0] == 7*168

PICK KNOWN DATA

In [18]:
known: DataFrame = test[(test['time'] > forecast_time) & (test['time'] <= pd.Timestamp(forecast_time) + pd.Timedelta(hours=12))]

In [19]:
known

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
31704,UP_MPNTLCDMRN_1,2020-08-13 23:00:00,0.848864,0.601757,0.933746,0.128603,-1.099286,-0.191301,-0.315362,0.027479,...,1321,UP_MPNTLCDMRN_1,1.661345,13,0.000959,8,0,2.749977,3,23
31705,UP_MPNTLCDMRN_1,2020-08-14 00:00:00,1.670850,0.639648,0.919851,0.075519,-1.128536,-0.191301,-0.255689,0.221590,...,1321,UP_MPNTLCDMRN_1,-1.661381,14,0.500952,8,0,2.750118,4,0
31706,UP_MPNTLCDMRN_1,2020-08-14 01:00:00,0.811447,0.090229,0.933746,0.146297,-1.109036,-0.191301,-0.196016,0.376880,...,1321,UP_MPNTLCDMRN_1,-1.516914,14,0.500952,8,0,2.750260,4,1
31707,UP_MPNTLCDMRN_1,2020-08-14 02:00:00,0.824715,-0.421299,0.878167,0.057825,-1.148035,-0.191301,-0.196016,0.415702,...,1321,UP_MPNTLCDMRN_1,-1.372448,14,0.500952,8,0,2.750401,4,2
31708,UP_MPNTLCDMRN_1,2020-08-14 03:00:00,0.195629,-0.838100,0.864272,-0.083731,-1.216285,-0.191301,-0.196016,0.415702,...,1321,UP_MPNTLCDMRN_1,-1.227982,14,0.500952,8,0,2.750542,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242095,UP_PRCLCMINEO_1,2020-08-14 06:00:00,-0.689231,0.003895,1.212904,-0.084582,-0.357503,-0.159763,0.042203,0.070870,...,1321,UP_PRCLCMINEO_1,-0.794583,14,0.500952,8,6,2.750967,4,6
242096,UP_PRCLCMINEO_1,2020-08-14 07:00:00,-0.694629,-0.413023,1.674744,-0.120673,-0.393239,-0.159763,0.042203,-0.183379,...,1321,UP_PRCLCMINEO_1,-0.650116,14,0.500952,8,6,2.751108,4,7
242097,UP_PRCLCMINEO_1,2020-08-14 08:00:00,-0.684232,-0.602531,1.949352,-0.084582,-0.339634,-0.159763,0.106010,-0.310504,...,1321,UP_PRCLCMINEO_1,-0.505650,14,0.500952,8,6,2.751249,4,8
242098,UP_PRCLCMINEO_1,2020-08-14 09:00:00,0.103824,-0.356170,2.149067,-0.084582,-0.321766,-0.159763,0.488851,0.028495,...,1321,UP_PRCLCMINEO_1,-0.361184,14,0.500952,8,6,2.751391,4,9


In [20]:
assert known.shape[0] == 12*7
assert len(known['time'].unique()) == 12

CONCAT THE TWO DF

In [21]:
inference_data: DataFrame = pd.concat([obs, known], axis=0, ignore_index=True)
inference_data: DataFrame = inference_data.sort_values(by=['plant_name_up','time'], ascending=True, ignore_index=True)

In [22]:
inference_data

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2020-08-06 23:00:00,-0.529162,0.866993,0.239003,-0.048342,-1.274784,-0.191301,-0.733072,-0.050166,...,1314,UP_MPNTLCDMRN_1,1.661345,6,0.000959,8,0,2.726227,3,23
1,UP_MPNTLCDMRN_1,2020-08-07 00:00:00,-0.576865,0.866993,0.197319,-0.101425,-1.304034,-0.191301,-0.733072,-0.127810,...,1314,UP_MPNTLCDMRN_1,-1.661381,7,0.500952,8,0,2.726368,4,0
2,UP_MPNTLCDMRN_1,2020-08-07 01:00:00,-0.642457,0.904884,0.127844,-0.119119,-1.304034,-0.191301,-0.792745,-0.321922,...,1314,UP_MPNTLCDMRN_1,-1.516914,7,0.500952,8,0,2.726510,4,1
3,UP_MPNTLCDMRN_1,2020-08-07 02:00:00,-0.666309,0.866993,0.072265,-0.136814,-1.313784,-0.191301,-0.792745,-0.399567,...,1314,UP_MPNTLCDMRN_1,-1.372448,7,0.500952,8,0,2.726651,4,2
4,UP_MPNTLCDMRN_1,2020-08-07 03:00:00,-0.691651,0.848048,0.044475,-0.136814,-1.323534,-0.191301,-0.792745,-0.360744,...,1314,UP_MPNTLCDMRN_1,-1.227982,7,0.500952,8,0,2.726793,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,UP_PRCLCMINEO_1,2020-08-14 06:00:00,-0.689231,0.003895,1.212904,-0.084582,-0.357503,-0.159763,0.042203,0.070870,...,1321,UP_PRCLCMINEO_1,-0.794583,14,0.500952,8,6,2.750967,4,6
1256,UP_PRCLCMINEO_1,2020-08-14 07:00:00,-0.694629,-0.413023,1.674744,-0.120673,-0.393239,-0.159763,0.042203,-0.183379,...,1321,UP_PRCLCMINEO_1,-0.650116,14,0.500952,8,6,2.751108,4,7
1257,UP_PRCLCMINEO_1,2020-08-14 08:00:00,-0.684232,-0.602531,1.949352,-0.084582,-0.339634,-0.159763,0.106010,-0.310504,...,1321,UP_PRCLCMINEO_1,-0.505650,14,0.500952,8,6,2.751249,4,8
1258,UP_PRCLCMINEO_1,2020-08-14 09:00:00,0.103824,-0.356170,2.149067,-0.084582,-0.321766,-0.159763,0.488851,0.028495,...,1321,UP_PRCLCMINEO_1,-0.361184,14,0.500952,8,6,2.751391,4,9


In [23]:
assert inference_data.shape[0] == 7*168 + 7*12

In [24]:
inference_data.columns

Index(['plant_name_up', 'time', 'kwh', 'dew_point_2m_C', 'temperature_2m_C',
       'msl_pressure_hPa', 'sfc_pressure_hPa', 'precipitation_1h_mm',
       'wind_speed_mean_10m_1h_ms', 'wind_speed_mean_100m_1h_ms',
       'wind_dir_mean_100m_1h_d', 'wind_dir_mean_10m_1h_d',
       'wind_gusts_10m_1h_ms', 'wind_gusts_100m_1h_ms', 'wind_gusts_100m_ms',
       'wind_gusts_10m_ms', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 't',
       'days_from_start', 'id', 'hour', 'day', 'day_of_week', 'month',
       'categorical_id', 'hours_from_start', 'categorical_day_of_week',
       'categorical_hour'],
      dtype='object')

PREPARE INFRASTRUCTURE FOR TRAINING

In [25]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [26]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [27]:
# Tensorflow setup
# default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [28]:
fixed_params: Dict = formatter.get_experiment_params()
fixed_params

{'total_time_steps': 180,
 'num_encoder_steps': 168,
 'num_epochs': 100,
 'early_stopping_patience': 10,
 'multiprocessing_workers': 5,
 'column_definition': [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>),
  ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>),
  ('kwh', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
  ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('hours_from_start',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.KNOWN_INPUT: 2>),
  ('dew_point_2m_C',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('temperature_2m_C',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('msl_pressure_hPa',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('sfc_pressure_hPa',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('precipitation_1h_mm',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSER

In [29]:
# # Sets up default params
# fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [30]:
params

{'dropout_rate': 0.1,
 'hidden_layer_size': 160,
 'learning_rate': 0.001,
 'minibatch_size': 64,
 'max_gradient_norm': 0.01,
 'num_heads': 4,
 'stack_size': 1,
 'model_folder': 'C:\\Users\\Lorenzo\\PycharmProjects\\TFT\\outputs\\saved_models\\sorgenia_wind\\fixed'}

In [31]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

*** Loading hyperparm manager ***


In [32]:
opt_manager.hyperparam_folder = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_mm+cop'
model_folder: str = opt_manager.hyperparam_folder
model_folder

'C:\\Users\\Lorenzo\\savedmodels_sorgenia_wind_mm+cop'

# PREDICTION

In [34]:
t0: float = time.perf_counter()
print("*** Computing Preds ***")
tf1.reset_default_graph()
with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
    tf1.keras.backend.set_session(sess)
    params: Dict = opt_manager.get_next_parameters()
    params['exp_name'] = 'sorgenia_wind'
    params['data_folder'] = config.data_csv_path
    model = TemporalFusionTransformer(params, use_cudnn=True)
    params.pop('exp_name', None)
    params.pop('data_folder', None)
    # load model
    model.load(model_folder, use_keras_loadings=False)
    
    output_map: Dict = model.predict(inference_data, return_targets=False)
    p50_forecast: DataFrame = formatter.format_predictions(output_map.get("p50"))

t1: float = time.perf_counter()
print("Time elapsed ", t1-t0)

*** Computing Preds ***
category counts are  [7]
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 160
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = C:\Users\Lorenzo\PycharmProjects\TFT\outputs\saved_models\sorgenia_wind\fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 180
# num_encoder_steps = 168
# num_epochs = 100
# early_stopping_patience = 10
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('kwh', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('dew_point_2m_C', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('temperature

Loading model from C:\Users\Lorenzo\savedmodels_sorgenia_wind_mm+cop\TemporalFusionTransformer.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Lorenzo\savedmodels_sorgenia_wind_mm+cop\TemporalFusionTransformer.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) Not found: Key TemporalFusionTransformer/cu_dnnlstm/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at ..\libs\utils.py:181) ]]
  (1) Not found: Key TemporalFusionTransformer/cu_dnnlstm/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at ..\libs\utils.py:181) ]]
	 [[save/RestoreV2/_1653]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "C:\Users\Lorenzo\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Lorenzo\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Lorenzo\PycharmProjects\TF

In [ ]:
p50_forecast